In [111]:
import numpy as np

DATA_DIR = 'F:\\technopolis\\data\\'
GRAPH_PATH = 'F:\\technopolis\\graph\\'
MODEL_DIR = 'F:\\technopolis\\model\\'
SAMPLE_FRAC = 0.01
BATCH_SIZE = 1_000_000

SVM_ALPHA_RANGE = np.fromfunction(lambda i: 1e-15 * pow(10, i), (15,), dtype=float)
SVM_MAX_ITER_RANGE = np.fromfunction(lambda i: 10 * (i + 1), (20,), dtype=float)

FEATURES = [
    'DeviceClass',
    'DeviceName',
    'DeviceBrand',
    'DeviceCpu',
    'DeviceCpuBits',
    'OperatingSystemClass',
    'OperatingSystemName',
    'OperatingSystemVersion',
    'OperatingSystemNameVersion',
    'OperatingSystemVersionBuild',
    'LayoutEngineClass',
    'LayoutEngineName',
    'LayoutEngineVersion',
    'LayoutEngineVersionMajor',
    'LayoutEngineNameVersion',
    'LayoutEngineNameVersionMajor',
    'AgentClass',
    'AgentName',
    'AgentVersion',
    'AgentVersionMajor',
    'AgentNameVersion',
    'AgentNameVersionMajor',
#     'from',
#     'to',
#     'url',
#     'requestType',
    'operation'
]

In [112]:
!pip install pyarrow
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import seaborn as sns
import user_agents
%matplotlib inline
from matplotlib import pyplot as plt
pd.set_option("display.precision", 2)

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.tree import export_graphviz
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.externals import joblib
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel

import os

import warnings
warnings.filterwarnings('ignore')

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Выбор фич

In [123]:
http = pq.read_table(DATA_DIR + 'HTTPRequests-20180217_1718_parsedUAA_parquet.parquet', columns=FEATURES ).to_pandas().sample(frac=SAMPLE_FRAC)
target = pq.read_table(DATA_DIR + 'HTTPRequests-20180217_1718_parsedUAA_parquet.parquet', columns=['isBot'] ).to_pandas().sample(frac=SAMPLE_FRAC)

In [124]:
# убираем None-ы
for f in FEATURES:
    http[f] = http[f].astype('str').fillna(f+'None')
    http[f] = f + '_' + http[f].astype('str')
    
# получаем все значения фич в виде матрицы
column_to_arrays = http.as_matrix(columns=FEATURES).transpose()
array_to_encode = column_to_arrays[0]

# объединяем в один np массив, чтобы прогнать в le
for i in range(1, len(column_to_arrays)):
    array_to_encode = np.concatenate((array_to_encode, column_to_arrays[i]), axis=0)

le = LabelEncoder()
le.fit(array_to_encode)

LabelEncoder()

In [125]:
# применяем le к значениям фич
X_le = http[FEATURES].apply(le.transform, axis=0)
y = target

In [126]:
ohe = OneHotEncoder(handle_unknown='ignore')
X_ohe = ohe.fit_transform(X_le)

In [127]:
clf = SGDClassifier(random_state=17, max_iter=1000, alpha=0.0000001, loss='log')
feature_selector = SelectFromModel(clf)

print("Shape before feature selection: {}".format(X_ohe.shape))
X_new = feature_selector.fit_transform(X_ohe, y)
print("Shape after feature selection: {}".format(X_new.shape))
print("Selected:")
selected = feature_selector.get_support(indices=True)
for f in selected:
    try:
        print(le.inverse_transform([f])[0])
    except ValueError:
        pass


Shape before feature selection: (10104, 2169)
Shape after feature selection: (10104, 434)
Selected:
AgentClass_1.0
AgentClass_1.6.689.40
AgentClass_10.1.1
AgentClass_10.9.8.770
AgentClass_11.0.2
AgentClass_15.12.1.6398
AgentClass_15.6.2311.5029
AgentClass_17.0.963.47
AgentClass_17.1.0.412.01
AgentClass_18.0.1025.166
AgentClass_30.0.1835.88
AgentClass_39.0.2256.71
AgentClass_4.4.0.600
AgentClass_40.0.2308.81
AgentClass_42.0.2311.152
AgentClass_42.0.2393.351
AgentClass_43.0.2357.134
AgentClass_43.0.2442.1144
AgentClass_45.0.2454.94
AgentClass_45.0.2552.25968
AgentClass_46.0.2490.80
AgentClass_47.0.2526.111
AgentClass_47.0.2526.35
AgentClass_47.0.2526.73
AgentClass_47.0.2526.80
AgentClass_47.0.2529.83
AgentClass_47.0.2545.34
AgentClass_47.5.2526.115
AgentClass_48.0.2685.50
AgentClass_49.0.2600.0
AgentClass_49.0.2623.112
AgentClass_50.0.2661.86
AgentClass_50.0.2661.89
AgentClass_53.0.2785.143
AgentClass_56.0.2924.197
AgentClass_56.0.2924.92
AgentClass_56.1.2924.71
AgentClass_56.1.2924.82
A